<a href="https://colab.research.google.com/github/ohmkas/Carteira-Quant-FI/blob/main/coleta_dados_cadastrais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from zipfile import ZipFile
import requests, zipfile, io
from os import chdir, getcwd, listdir
import numpy as np
import pandas as pd
from datetime import date, datetime, timedelta
import time
from os import chdir,listdir

def salvar(nome_pasta=str): #Cria ess pasta caso ela não exista ou simplemente já deixa o caminho da pasta pronto
  if not os.path.exists("/content/sample_data/"+nome_pasta): 
      os.mkdir("/content/sample_data/"+nome_pasta) 
  caminho = str("/content/sample_data/"+nome_pasta) 
  print('os arquivos serão salvos em:',caminho)
  return(caminho)

salvar_em = salvar(nome_pasta='Carteira Quant')

inicio_coleta = (date.today()-timedelta(60)).strftime('%Y-%m')

def consultar_dados_cadastrais_cvm():
  dados_cadastrais = pd.read_csv('http://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv', 
                       sep=';', encoding='ISO-8859-1',
                       index_col='CNPJ_FUNDO',low_memory=False) 
                      
  dados_cadastrais = dados_cadastrais.loc[dados_cadastrais['SIT'] != 'CANCELADA'] #apenas fundos ativos, em funcionamento
  print('requisitando download : http://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv')
  return(dados_cadastrais)

dados_cadastrais_fis = consultar_dados_cadastrais_cvm()

def consultar_informações_cadastrais_específicos(link=str,
                                                 nome_arquivo=str,
                                                 nome_pasta=str,
                                                 somente_ultimo_mês=True):
  if somente_ultimo_mês==True:
    período_coletado = (date.today()-timedelta(60)).strftime('%Y%m')
    data = período_coletado
    print('requisitando download : ',link+nome_arquivo+data+'.zip')
    url = link+nome_arquivo+data+'.zip'
    download = requests.get(url) 
    extração = zipfile.ZipFile(io.BytesIO(download.content))  
    extração.extractall(salvar_em+'/'+nome_pasta)
    caminho_pasta = salvar_em+'/'+nome_pasta


  else:
    período_coletado = pd.date_range(inicio_coleta, (date.today()-timedelta(60)).strftime('%Y%m'), freq='M')
    for data in período_coletado:
      data = data.strftime('%Y%m')
      print('requisitando download : ',link+nome_arquivo+data+'.zip')
      url = link+nome_arquivo+data+'.zip'
      download = requests.get(url) 
      extração = zipfile.ZipFile(io.BytesIO(download.content))  
      extração.extractall(salvar_em+'/'+nome_pasta)
      caminho_pasta = salvar_em+'/'+nome_pasta
  return (caminho_pasta)

def ler_arquivo_específico(caminho_arquivo=str):
  archive = pd.read_csv(caminho_arquivo,
            sep=';', 
            encoding='ISO-8859-1',
            error_bad_lines=False,
            warn_bad_lines=False,
            index_col='CNPJ_FUNDO')
  return archive

def arquivos_extraídos(funcao_consultar_informações_cadastrais_específicos):
  arquivos_baixados = {}
  chdir(funcao_consultar_informações_cadastrais_específicos)
  for arquivo in listdir():
    arquivo_lido = ler_arquivo_específico(arquivo)
    arquivos_baixados.update({arquivo:arquivo_lido})
  return(arquivos_baixados)
  
dados_lâmina_fis = arquivos_extraídos(consultar_informações_cadastrais_específicos(link='http://dados.cvm.gov.br/dados/FI/DOC/LAMINA/DADOS/',
                                             nome_arquivo='lamina_fi_',
                                             nome_pasta='lâmina_fi'),)
dados_complementares_fis = ler_arquivo_específico('http://dados.cvm.gov.br/dados/FI/DOC/PERFIL_MENSAL/DADOS/perfil_mensal_fi_'+(date.today()-timedelta(60)).strftime('%Y%m')+'.csv')

dados_extrato_fis = (ler_arquivo_específico('http://dados.cvm.gov.br/dados/FI/DOC/EXTRATO/DADOS/extrato_fi_'+(date.today()-timedelta(60)).strftime('%Y')+'.csv')).dropna(axis=1)

dados_suplementares_fis = {'Cadastrais':dados_cadastrais_fis,
                       'Lâmina': dados_lâmina_fis['lamina_fi_' + str((date.today()-timedelta(60)).strftime('%Y%m')) + '.csv'],
                       'Lâmina Carteira':dados_lâmina_fis['lamina_fi_'+ str((date.today()-timedelta(60)).strftime('%Y%m'))+ '.csv'], #inserir aqui a lamina segregada lamina[tal] e outra chave para outra lamina[tal]
                       'Extrato': dados_extrato_fis,
                       'Complementares': dados_complementares_fis}

for key in dados_suplementares_fis:
  dados_suplementares_fis[key].to_csv(salvar_em+'/'+key+'.csv')

os arquivos serão salvos em: /content/sample_data/Carteira Quant
requisitando download : http://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv
requisitando download :  http://dados.cvm.gov.br/dados/FI/DOC/LAMINA/DADOS/lamina_fi_202109.zip
